In [2]:
# read test data
import pandas as pd
import numpy as np

data = pd.read_csv("../dataset/hlab/hlab_test.csv")
data["id"] = list(range(1, data.shape[0]+1))

cols = data.columns.tolist()
cols.insert(0, cols.pop(cols.index('id')))
data = data.reindex(columns=cols)

print(data.head(2))
print(data.shape)

data.to_csv("../dataset/hlab/hlab_test2.csv", index=False)

   id          HLA   peptide  Label  Length  \
0   1  HLA-A*01:01  LFGRDLSY      1       8   
1   2  HLA-A*01:01  TDKKTHLY      1       8   

                                  mhc  
0  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
1  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
(172580, 6)


In [4]:
# group by peptide length (k-mer)
data_grouped = []  # cada elemento es un dataframe. 
# Ex: data_list[0] contiene los datos de peptidos de longitud 8
# Ex: data_list[1] contiene los datos de peptidos de longitud 9

for i in range(8,15):
    peptides = data[data['Length'] == i] 
    data_grouped.append(peptides)
    #peptides.to_csv(str(i) + "-mer.csv", index=False)

print(data_grouped[0].head(3)) # 8-mer peptides

   id          HLA   peptide  Label  Length  \
0   1  HLA-A*01:01  LFGRDLSY      1       8   
1   2  HLA-A*01:01  TDKKTHLY      1       8   
2   3  HLA-A*01:01  RSDTPLIY      1       8   

                                  mhc  
0  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
1  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
2  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  


In [6]:
# por cada grupo k-mer, ahora separamos por el tipo de HLA
import os
import glob

def predict_anthem(hla, peptides):
    # save peptides in a tmp file
    tmp_peptide_file = open("temp_peptide_file.txt", "w")
    tmp_peptide_file.write("\n".join(peptides))
    tmp_peptide_file.close()

    length = str(len(peptides[0]))

    # cmd for anthem
    #python sware_b_main.py --length 9 --HLA HLA-A*01:01 --mode prediction --peptide_file test/predictpeptide.txt

    cmd = "cd Anthem; python sware_b_main.py --length "+length+" --HLA "+hla+" --mode prediction --peptide_file ../temp_peptide_file.txt"    
    os.system(cmd)
    results = get_last_folder_created()    
    files = glob.glob("Anthem/" + str(results) + "/*.txt")
    anthem_results = open(files[0], "r")
    lines = anthem_results.readlines() 
    lines = lines[5:len(lines)-3] # remove header and bottom
    bindings = []
    probs = []
    for line in lines: # procesamos cada peptido 
        tmp = line.split("\t")
        #print(tmp)
        peptide = tmp[3] # aqui esdta el peptido
        binding = 1 if tmp[5].strip() == "yes" else 0 # aqui esta si is binding
        prob = float(tmp[7].replace("\n", "").strip())
        
        bindings.append(binding)
        probs.append(prob)
        #print(peptide, binding)
    return bindings , probs  
    
    
    

# anthem genera los resultados en un folder, con esta funcion detectamos ese folder
def get_last_folder_created():
    folder = 'Anthem'
    possible_folder = [] # subfolder con nombre de fechas
    sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
    for folder in sub_folders:
        if folder.isnumeric():
            possible_folder.append(int(folder))
    return max(possible_folder)

  
final_results = pd.DataFrame(columns=['id','HLA','peptide','Label','Length','mhc','anthem_pred', 'anthem_prob'])
for df_by_kmer in data_grouped: # itereamos por cada k-mer
    print("generating...")
    hlas = df_by_kmer['HLA'].unique()
    for hla in hlas:
        df_by_hla = df_by_kmer[df_by_kmer['HLA'] == hla] # dividimos por tipo de hla
        peptides = df_by_hla["peptide"].tolist()     
        bindings, probs = predict_anthem(hla, peptides)   
        df_by_hla["anthem_pred"] = bindings
        df_by_hla["anthem_prob"] = probs
        final_results = pd.concat([final_results, df_by_hla]) 

        
    

#final_results.to_csv("results_tmp.csv", index=False)
print("finish :)")

generating...


/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_pred"] = bindings
/tmp/ipykernel_8630/469941303.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_prob"] = probs
/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

generating...


/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_pred"] = bindings
/tmp/ipykernel_8630/469941303.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_prob"] = probs
/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

generating...


...

generating...


/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_pred"] = bindings
/tmp/ipykernel_8630/469941303.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_prob"] = probs
/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

generating...


/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_pred"] = bindings
/tmp/ipykernel_8630/469941303.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_hla["anthem_prob"] = probs
/tmp/ipykernel_8630/469941303.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

finish :)


In [7]:
final_results = final_results.sort_values('id')
final_results.to_csv("prediction_anthem.csv", index=False)

# Prediction with ESMt6

Vamos a aplicar softmax para normalizar los logits obtenidos en las predicciones de esm2t6 y tape

In [1]:
import pandas as pd
import numpy as np

def softmax(logits):
    return (np.exp(logits) / np.exp(logits).sum() )

#print(softmax([16.917, 0.772]))

data = pd.read_csv("../predictions/30epochs_esm2_t6_freeze.csv", index_col=0)
data['prob_0'] = data.apply(lambda row: ( softmax([row[0], row[1]])[0] ), axis=1)
data['prob_1'] = data.apply(lambda row: ( softmax([row[0], row[1]])[1] ), axis=1)
data.to_csv("../predictions/30epochs_esm2_t6_freeze_softmax.csv")


In [2]:
data = pd.read_csv("../predictions/30epochs_tape_acc_steps.csv", index_col=0)
data['prob_0'] = data.apply(lambda row: ( softmax([row[0], row[1]])[0] ), axis=1)
data['prob_1'] = data.apply(lambda row: ( softmax([row[0], row[1]])[1] ), axis=1)
data.to_csv("../predictions/30epochs_tape_acc_steps_softmax.csv")

In [9]:
print(data.head(10))

          0         1  prediction  label    prob_0    prob_1
0 -1.515012  2.123430           1      1  0.025620  0.974380
1 -2.883986  3.783111           1      1  0.001270  0.998730
2 -4.096594  5.074096           1      1  0.000104  0.999896
3 -3.912578  5.064070           1      1  0.000126  0.999874
4 -1.187903  2.257972           1      1  0.030892  0.969108
5  0.383407  0.175635           0      1  0.551757  0.448243
6 -1.899285  2.613402           1      1  0.010850  0.989150
7 -2.634839  3.608717           1      1  0.001939  0.998061
8 -1.740799  2.500628           1      1  0.014183  0.985817
9 -2.192737  3.056245           1      1  0.005225  0.994775
